# Importing libraries

In [13]:
import trimesh as tm
import os
import pyvista as pv
import numpy as np
import topogenesis as tg

# loading lattice and context, setting default values

In [14]:
lattice_path = os.path.relpath("../data/voxelized_envelope.csv")
occ_lattice = tg.lattice_from_csv(lattice_path) # a default occupation lattice

path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height, will be USER INPUT???)

base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint <-- easier way?
plot_area = base.area/2 # area calculates both sides of the flattened mesh, we need only 1

env_all_vox = occ_lattice.flatten() # flattened lattice of all possible positions, True is occupied, False is unoccupied
vox_size = env_all_vox.unit[0] 

# FSI

Global objective function



In [52]:
def FSI(m2_plot, occupationlattice): # FSI based on the plot Area & all occupied cells in the lattice, for 2nd stage
    cell_count = sum(i for i in occupationlattice.flatten() if i != False) # <-- maybe rewrite? generator expression possible?
    m2_total = vox_size**2*cell_count
    FSI = m2_total/m2_plot
    return FSI

def FSI_lowres(floors, floor_percent=0.75): # quick FSI based on floorcount & plot coverage of the floors, for 1st stage
    FSI = floor_percent*floors
    return FSI

FSI(plot_area, occ_lattice)

3.056270794229774

# PV Yield

Global objective function

In [41]:
# Annual solar energy output of PV is calculated as follows:
# E(kwh) = A*r*H*PR where A = total solar panel area, r = solar panel yield/efficiency, H = Annual average solar radiation, PR = performance ratio
# from: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 

r = 0.15 # yield/efficiency estimate
H = 1050 # to be updated, currently wikipedia estimate. Slope, tilt, orientation may change values. kWh.y so maybe *365???
PR = 0.75 # default value, estimate

In [48]:
def PV(occupationlattice): # an accurate value (kWh estimate) of the PV potential of the mass <-- this currently uses uniform data for r, H, PR. 
    occ_shifted_down = np.roll(occupationlattice, (0,0,-1), (0,1,2)) # TODO: check calculation method, seems plausible
    roof_mask = (occ_lattice == True)*(occ_shifted_down == False)
    roof_mask_lattice = tg.to_lattice(roof_mask, occ_lattice)
    cell_count = sum(i for i in roof_mask_lattice.flatten() if i != False) # <-- maybe rewrite? generator expression possible?
    A = cell_count*vox_size
    E = A*r*H*PR
    return E

def PV_lowres(floors): # this gives the ratio between roof area and total area, for 1st stage
    PV = 1/floors
    return PV

PV(occ_lattice)

217350.0

# Daylighting

Local objective function

# Zoning

Local cost function